In [23]:
#Preprocessing Images for tensorflow CNN model
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

#Define dimensions for images
img_height = 227
img_width = 227
batch_size = 50

img_dir = "/home/yodazon/python/Project_3DPrintingApp/CNNBuilding/images_combined"

# Get list of all image file paths
all_image_paths = [os.path.join(img_dir, fname) for fname in os.listdir(img_dir)]

# Get list of all image labels (assuming filenames are label_image_number.jpg)
all_image_labels = [fname.split('_')[0] for fname in os.listdir(img_dir)]

In [24]:
# Split data into training, validation, and test sets
train_files, test_files, train_labels, test_labels = train_test_split(all_image_paths, all_image_labels, test_size=0.2, random_state=42)
train_files, val_files, train_labels, val_labels = train_test_split(train_files, train_labels, test_size=0.2, random_state=42)

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_files, test_labels))




In [27]:
# Function to preprocess images
#CODE FROM CHAT GPT
def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img, label

#CODE FROM KAGGLE
def process_image(image,label):
    image=tf.image.per_image_standardization(image)
    image=tf.image.resize(image,(64,64))
    
    return image,label

In [26]:
# Apply preprocessing and batching
train_ds = train_ds.map(preprocess_image).batch(batch_size)
val_ds = val_ds.map(preprocess_image).batch(batch_size)
test_ds = test_ds.map(preprocess_image).batch(batch_size)

In [17]:
#Defining tensor flow model
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10,activation='softmax')  
    
    
])

In [18]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.optimizers.SGD(learning_rate=0.001),
    metrics=['accuracy']    
)
model.summary()
tf.config.list_physical_devices('GPU')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 55, 55, 128)       46592     
                                                                 
 batch_normalization_10 (Ba  (None, 55, 55, 128)       512       
 tchNormalization)                                               
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 27, 27, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 27, 27, 256)       819456    
                                                                 
 batch_normalization_11 (Ba  (None, 27, 27, 256)       1024      
 tchNormalization)                                               
                                                      

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# Step 4: Train your model
X_train, X_val, y_train, y_val = train_files, val_files, train_labels, val_labels 

history = model.fit(train_ds,batch_size=50,shuffle=True, epochs=10, validation_data=(val_ds))

Epoch 1/10


2024-03-15 19:22:55.129025: W tensorflow/core/framework/op_kernel.cc:1816] OP_REQUIRES failed at cast_op.cc:122 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node Cast_1 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/yodazon/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/yodazon/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/yodazon/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/yodazon/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/yodazon/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in dis